# Data Discovery Studio Jupyter Notebook Dispatcher

## 1. Execute the cells below to get parameters from DDS and select a notebook for processing ###

Example call with Document ID from DDS: CinergiDispatch.ipynb?documentId=61cc7f6afb5246d2be41811e94a1a8ea


In [5]:
# Check if the parameters are correct
from __future__ import print_function
import ntpath
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import webbrowser
from lxml import etree  #supposed to be better than xml.etree

documentID=''

In [6]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("documentID='".concat(getQueryStringValue("documentId")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

<IPython.core.display.Javascript object>

## 2. Print out parameters passed to Jupyter: ##

In [7]:

# use hardwired values for testing
# these only get used if no documentID is passes as an argument when 
#   opening the notebook (per getQueryStringValue javascript function at start)
catalogURL = "http://datadiscoverystudio.org/geoportal/"
if (len(documentID)==0):
    #documentID="e3619c5df2644204b67f51f48525a0b1"    #NGDS wfs
    #documentID="4db8156abb6d4119aa5c35aa39514b42"   #sciencebase WFS
    #documentID="61cc7f6afb5246d2be41811e94a1a8ea"   #ndbc data
    documentID="de5383bf941d4d60ae9443bd7ffa9a33"   #Magic data
    #documentID="b20f8f12ef594520abb0e5efbcd891fe"   #nwis qwdata

In [8]:

url_partitioned = full_notebook_url.partition('/CinergiDispatch')
base_url = url_partitioned[0];

print("User: ",user)
print("DocumentID: ", documentID)
# print("full notebook url partition", url_partitioned)
print("Full notebook URL: ", full_notebook_url)

User:  
DocumentID:  de5383bf941d4d60ae9443bd7ffa9a33
Full notebook URL:  http://localhost:8889/notebooks/jupyter-dispatchtests/ShutongCinergiDispatch.ipynb


---------------------------------------------------------------------------------------------------------------------

### New stuff

In [9]:
import pandas as pd
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import requests

#construct url for metadata
catalogURL = "http://datadiscoverystudio.org/geoportal/"
metadataURLx=catalogURL + 'rest/metadata/item/' + documentID + '/xml'
metadata = requests.get(metadataURLx)

#### parsing the xml metadata into a dict
P.S. Repetitive keys have its key value pair stored in a list at its level of hiearchy in the dict

In [10]:
#!pip install xmltodict

In [11]:
import xmltodict
from collections import OrderedDict

metadata_dict = xmltodict.parse(metadata.text)

In [12]:
#set up namespace map for ISO metadata
NSMAP = {"gmi":"http://www.isotc211.org/2005/gmi" ,
    "gco":"http://www.isotc211.org/2005/gco" ,
    "gmd":"http://www.isotc211.org/2005/gmd" ,
    "gml":"http://www.opengis.net/gml" ,
    "gmx":"http://www.isotc211.org/2005/gmx" ,
    "gts":"http://www.isotc211.org/2005/gts" ,
    "srv":"http://www.isotc211.org/2005/srv" ,
    "xlink":"http://www.w3.org/1999/xlink"}

In [13]:
#tree is an element tree
tree = etree.parse(metadataURLx)
#root = etree.tostring(tree.getroot())
root = tree.getroot()
docinfo = tree.docinfo
print(docinfo.xml_version)

1.0


In [14]:
title=tree.find("//gmd:identificationInfo//gmd:title/gco:CharacterString",namespaces=NSMAP).text

In [15]:
#title of the dataset, not sure how useful
#title = metadata_dict['gmi:MI_Metadata']['gmd:identificationInfo']['gmd:MD_DataIdentification']['gmd:citation']['gmd:CI_Citation']['gmd:title']['gco:CharacterString']

In [25]:
for  dist in tree.getiterator("{http://www.isotc211.org/2005/gmd}MD_Distribution"):
    if dist.find("gmd:distributionFormat/gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP) is not None: 
        # this code is not handling multiple formats under a distribution at this point. 
        # have to consider how to deal with binding between formats and digital transfer options
        dist_format = dist.find("gmd:distributionFormat/gmd:MD_Format/gmd:name/gco:CharacterString",namespaces=NSMAP).text
    else:
        dist_format = ''
    print(distribute_format)
    if dist.find("gmd:distributor/gmd:MD_Distributor//gmd:organisationName/.",namespaces=NSMAP) is not None:
        distorg =dist.find("gmd:distributor/gmd:MD_Distributor//gmd:organisationName/.",namespaces=NSMAP)
    else:
        distorg = ''
        
    if dist.find("gmd:distributor/gmd:MD_Distributor//gmd:MD_DigitalTransferOptions//gmd:CI_OnlineResource.",namespaces=NSMAP) is not None:
        distonline =dist.find("gmd:distributor/gmd:MD_Distributor//gmd:MD_DigitalTransferOptions//gmd:CI_OnlineResource",namespaces=NSMAP)
    else:
        distonline = ''

ASCII


In [ ]:
#subtree involving meta data distribution information

# this is problematic because it depends on use of the standard namespace prefixes, which
#  is not required by xml
md_dist = metadata_dict['gmi:MI_Metadata']['gmd:distributionInfo']['gmd:MD_Distribution']
distribute_format = md_dist['gmd:distributionFormat']['gmd:MD_Format']['gmd:name']['gco:CharacterString']
distributor = md_dist['gmd:distributor'][0]['gmd:MD_Distributor']['gmd:distributorContact']['gmd:CI_ResponsibleParty']['gmd:organisationName']['gco:CharacterString']
dist_landing = md_dist['gmd:distributor'][0]['gmd:MD_Distributor']['gmd:distributorTransferOptions']['gmd:MD_DigitalTransferOptions']['gmd:onLine']['gmd:CI_OnlineResource']['gmd:linkage']

#### Map the properties of metadata to suggested tokens

In [ ]:
#look up the combination of metadata features in a table and map the combination to potential tokens
def md_to_token(df, *features):
    #look up the combination of the features
    #precondition : features comes in order, if feature for corresponding column does not exist, leave it as null
    featurelist = list(features)
    assert len(featurelist) <= (df.shape[1] - 1), 'feature space dimension mismatch'
    sets = []
    col_indx = 0
    non_empty_sets = 0
    for feature in featurelist:
        index_set = set(df.loc[df.iloc[:, col_indx] == feature].index)
        sets += [index_set]
        if (len(index_set) > 0):
            non_empty_sets += 1
        col_indx = col_indx + 1
    
    #if the feature set is complete
    if non_empty_sets == df.shape[1] - 1:
        idx = list(set.intersection(*sets))
        return list(df['tokens'].loc[idx])
    
    #the feature set is not complete, suggest all possibilities
    else:
        idx = list(set.union(*sets))
        return list(df['tokens'].loc[idx])

#### Map tokens to the notebook url

In [ ]:
#map the given set to tokens to a set of urls
def token_to_url(tokens, df):
    #tokens is a list of string tokens
    #assume the token space of this url-mapping df is the same as the one of the token mapping df
    urls = list(df.loc[df['tokens'].apply(lambda x: x in tokens)]['url'])
    return urls

In [ ]:
#not really used
def token_to_desc(tokens, df):
    #df is a mapping table from tokens to labels and descriptions
    #return a dict with keys as tokens and values as another dict - 
    #whose keys are descprtion and label and values are corresponding values
    subset = df.loc[df['tokens'].apply(lambda x: x in tokens)].set_index('tokens')
    return subset.to_dict('index')

#### Demo, for testing.

an example token map


In [ ]:
md_token_map = pd.DataFrame({'feature1': ['a', 'b', 'c', 'b'],
                             'feature2': ['d', 'e', 'f', 'x'],
                             'feature3': ['g', 'h', 'i', 'y'],
                             'tokens': [1, 2, 3, 4]})
md_token_map

In [ ]:
token_url_map = pd.DataFrame({'tokens': [1, 2, 3, 4],
                             'url': ['url1', 'url2', 'url3', 'url4']})
token_url_map

In [ ]:
token_desc_map = pd.DataFrame({'tokens': [1, 2, 3, 4],
                              'label': ['label1', 'label2', 'label3', 'label4'],
                              'description': ['desc1', 'desc2', 'desc3', 'desc4']})
token_desc_map

In [ ]:
url_desc_map = pd.merge(left = token_url_map, right = token_desc_map, on = 'tokens')[['url', 'label', 'description']].set_index('url')
url_desc_map

suppose I attain feature combination of [b, e, h]. Actual datasets should have more combinations 

In [ ]:
feature1 = 'b'
feature2 = 'e'
feature3 = 'h'
tokens = md_to_token(md_token_map, feature1, feature2, feature3)
urls = token_to_url(tokens, token_url_map)
token_info = token_to_desc(tokens, token_desc_map)

test_menu = OrderedDict()
for url in urls:
    menu_label = dict(url_desc_map.loc[url])['label']
    if menu_label in test_menu.keys():
        menu_label = menu_label + '1'
    test_menu[menu_label] = url
def wrapper(menu):
    return menu
test_out = interact(wrapper, menu = test_menu)

Another example that returns >1 url

In [ ]:
import numpy as np

In [ ]:
feature4 = 'b'
feature5 = np.nan
feature6 = np.nan
tokens2 = md_to_token(md_token_map, feature4, feature5, feature6)
urls2 = token_to_url(tokens2, token_url_map)

test_menu2 = OrderedDict()
for url in urls2:
    menu_label = dict(url_desc_map.loc[url])['label']
    if menu_label in test_menu2.keys():
        menu_label = menu_label + '1'
    test_menu2[menu_label] = url
def wrapper(menu):
    return menu
test_out2 = interact(wrapper, menu = test_menu2)

----------------------------------------------------------------------------------------------------------------------------

## 4. Open the URL of the selected notebook ##

In [ ]:
chosen_nb_name = nb_menu[out.widget.result]
url1 = ('{base_url}/operations/{nb_name}?'+'documentId='+documentID+'&'+'user='+user).format(base_url=base_url, nb_name=chosen_nb_name)

#webbrowser.open(url1)
webbrowser.open_new(url1)
print("CLICK TO OPEN THE URL: ", url1)
